In [34]:
import random
import numpy as np


from numpy import pi as π

import perceval as pv
from scipy.optimize import minimize
from perceval.components import *

from collections import Counter

In [35]:
def addPrep(circuit, prepTheta, prepPhi):

    circuit.add(1, BS.H())
    circuit.add(1, PS(prepTheta/2))
    circuit.add(1, BS.H())
    circuit.add(2, PS(prepPhi))

def deAddPrep(circuit, prepTheta, prepPhi):
    circuit.add(2, PS(-prepPhi))
    circuit.add(1, BS.H())
    circuit.add(1, PS(-prepTheta/2))
    circuit.add(1, BS.H())

def removePrep(circuit, prepTheta, prepPhi):
    circuit.add(0, PS(-prepTheta/2))
    circuit.add(0, BS.H())
    circuit.add(0, PS(-prepPhi))
    circuit.add(0, BS.H())

    circuit.add(2, PS(-prepTheta/2))
    circuit.add(2, BS.H())
    circuit.add(2, PS(-prepPhi))
    circuit.add(2, BS.H())


def addEvolution(circuit, evTheta = None):
    # if evTheta == None:
    #     evTheta = [ pv.P(f'Phi_{i}') for i in range(12) ]

    circuit.add(0, BS.H())
    circuit.add(2, BS.H())
    
    circuit.add(0, PS(evTheta[0]))
    circuit.add(2, PS(evTheta[1]))

    circuit.add(0, BS.H())
    circuit.add(2, BS.H())
    
    circuit.add(1, PS(evTheta[2]))
    circuit.add(3, PS(evTheta[3]))

    circuit.add(1, BS.H())
    circuit.add(1, PS(evTheta[4]))
    circuit.add(1, BS.H())

    circuit.add(0, PS(evTheta[5]))
    circuit.add(2, PS(evTheta[6]))

    circuit.add(0, BS.H())
    circuit.add(2, BS.H())

    circuit.add(0, PS(evTheta[7]))
    circuit.add(2, PS(evTheta[8]))

    circuit.add(0, BS.H())
    circuit.add(2, BS.H())

    circuit.add(1, PS(evTheta[9]))
    circuit.add(3, PS(evTheta[10]))

    circuit.add(1, BS.H())
    circuit.add(1, PS(evTheta[11]))
    circuit.add(1, BS.H())


# def addMeasure(circuit, )

def getCircuit(evolTheta, prepTheta = 0, prepPhi = π):
    c = pv.Circuit(4)
    
    addPrep(c, prepTheta=prepTheta, prepPhi=prepPhi)
    c.barrier()
    addEvolution(c, evolTheta)
    c.barrier()
    removePrep(c, prepTheta=prepTheta, prepPhi=prepPhi)
    
    return c


def getCircuitTest(prepTheta = 0, prepPhi = π/2):
    c = pv.Circuit(4)
    
    addPrep(c, prepTheta=prepTheta, prepPhi=prepPhi)
    deAddPrep(c, prepTheta=prepTheta, prepPhi=prepPhi)
    
    return c

def getPsi(prepTheta = 0, prepPhi = π):
    c = pv.Circuit(4)
    addPrep(c, prepTheta=prepTheta, prepPhi=prepPhi)
    return c


In [36]:
# sv = pv.StateVector([0,1,0,1]) # From the paper, |0> up and |0> down


# circuit = getCircuit(prepTheta=π, prepPhi=π)


# stepper = pv.simulators.Stepper(pv.backends.SLOSBackend())
# stepper.set_circuit(circuit)


# for r, c in circuit:
#     sv = stepper.apply(sv, r, c)

# pv.pdisplay(circuit)
# for fockState, prob in sv:
#     if fockState[0] != fockState[1] and fockState[2] != fockState[3]: # Post-select only 
#         print(fockState, prob)

In [37]:
# c = Counter()
# for s in sv.samples(10000):
#     if s[0] != s[1] and s[2] != s[3]: #Since we introduce only 2 photons in the circuit it is enough
#         c[s] += 1
# print(c)

# # print(sv.samples(10000)[0])

In [38]:
# Fraction of [1, 0, x, y] states
def getFidelityRho1(counts):
    totCounts, validCounts = 0, 0

    for state in counts:
        totCounts += counts[state]
        if state[0] == 1 and state[1] == 0:
            validCounts += counts[state]

    return validCounts/totCounts


# Fraction of [x, y, 1, 0] states
def getFidelityRho2(counts):
    totCounts, validCounts = 0, 0

    for state in counts:
        totCounts += counts[state]
        if state[2] == 1 and state[3] == 0:
            validCounts += counts[state]

    return validCounts/totCounts


def costFunctionAtPhi(counts):
    return (1 - getFidelityRho1(counts))**2 + (1 - getFidelityRho2(counts))**2 + (getFidelityRho1(counts)-getFidelityRho2(counts))**2

In [39]:
def loss_function(theta = [0] * 12):
    cost = 0
    Phi = [ 0, π/2, π, 3*π/2 ]

    # Evaluate the 4 cost functions, one for each fidelity
    for phi_i in Phi:
        circuit = getCircuit(evolTheta=theta, prepTheta=π/2, prepPhi=phi_i)
        p = pv.Processor("SLOS", circuit)
        p.with_input(pv.BasicState([0, 1, 0, 1]))

        # The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
        sampler = pv.algorithm.Sampler(p)  
        sample_count = sampler.sample_count(1000)['results']

        # print(sample_count['results'])
        # sv = pv.StateVector([0,1,0,1])
        # circuit = getCircuit(evolTheta=theta, prepTheta=π/2, prepPhi=phi_i)

        # stepper = pv.simulators.Stepper(pv.backends.SLOSBackend())
        # stepper.set_circuit(circuit)

        

        # for r, c in circuit:
        #     sv = stepper.apply(sv, r, c)
        # for c in sample_count:
        #     print(c, sample_count[c])
        # counts = Counter()
        # for s in sv.samples(10000):
        #     if s[0] != s[1] and s[2] != s[3]: #Since we introduce only 2 photons in the circuit it is enough
        #         counts[s] += 1

        cost += costFunctionAtPhi(sample_count)
        
    return cost

In [ ]:
loss_function()

3.2823080000000004

In [41]:
def optimCallback(intermediate_result):
    
    outLine = f'{intermediate_result.fun:0.3f}, '
    for x in intermediate_result.x:
        outLine += f'{x}, '
    outLine += '\n'

    print(outLine)
    with open('ResultFile.csv', 'a+') as outFile:
        outFile.writelines([outLine])

In [ ]:
initialState = (np.random.rand(1, 12)[0])*2*π
initialState = [0]*12

with open('ResultFile.csv', 'w') as outFile:
    outFile.writelines('Fun, p0, p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11')

res = minimize(loss_function,
    initialState,
    method="Nelder-Mead",
    bounds=[(0, 2*π)]*12,
    callback=optimCallback
)

optimCallback(res)

3.319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0, 

3.319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0, 

3.319, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00025, 0.0, 0.0, 0.0, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-06, 7.089120370370375e-06, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-06, 7.089120370370375e-06, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-06, 7.089120370370375e-06, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 7.089120370370375e-06, 

3.294, 7.089120370370375e-06, 0.0, 7.089120370370375e-06, 7.8125e-0

In [45]:
res.fun

3.252012

# VQC

In [44]:
# input = pv.BasicState([0, 1, 0, 1])
# backend = pv.SLOSBackend()

# circuit = getCircuit(prepTheta=π, prepPhi=π)

# param_circuit = circuit.get_parameters()
# params_init = [random.random()*π for _ in param_circuit]


# # Run the optimisation
# o = optimize.minimize(loss_function, params_init, method="Powell")

# print(f"The maximum probability is {-loss_function(o.x)}")

# # For n=4, the probability should be 3/32
# # The maximum can also be obtained with the Hadamard matrix :

# H4 = (1/2)*np.array([[1,1,1,1], [1,-1,1,-1], [1,1,-1,-1], [1,-1,-1,1]])
# backend.set_circuit(pv.Unitary(pv.Matrix(H4)))
# backend.set_input_state(input)
# backend.probability(output_to_max)

# pv.pdisplay(circuit)